In [1]:

import warnings
warnings.filterwarnings('ignore')
# Import libraries and set desired options
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns

import os
import pickle
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from scipy.sparse import hstack
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV, TimeSeriesSplit
from sklearn.metrics import accuracy_score, f1_score

In [2]:
path='catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2'

In [3]:
import time
import datetime
from datetime import datetime, timedelta

In [4]:
%%time
times = ['time%s' % i for i in range(1, 11)]
sites = ['site%s' % i for i in range(1, 11)]
train_df = pd.read_csv(os.path.join(path, 'train_sessions.csv'), index_col='session_id')
test_df = pd.read_csv(os.path.join(path, 'test_sessions.csv'), index_col='session_id')
train_df = train_df.sort_values(by='time1')

y = train_df['target']
train_df.fillna(0, inplace = True)
test_df.fillna(0, inplace = True)
test_df[times] = test_df[times].astype('datetime64')
train_df[times] = train_df[times].astype('datetime64')
y.astype('int8').values
y

Wall time: 18.7 s


session_id
21669     0
54843     0
77292     0
114021    0
146670    0
         ..
12224     0
164438    0
12221     0
156968    0
204762    0
Name: target, Length: 253561, dtype: int64

In [5]:
test_sessions =  test_df.loc[:,(sites)]
train_sessions = train_df.loc[:,(sites)]
test_times =  test_df.loc[:,(times)]
train_times = train_df.loc[:,(times)]
train_sessions

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
21669,56,55.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
54843,56,55.0,56.0,55.0,0.0,0.0,0.0,0.0,0.0,0.0
77292,946,946.0,951.0,946.0,946.0,945.0,948.0,784.0,949.0,946.0
114021,945,948.0,949.0,948.0,945.0,946.0,947.0,945.0,946.0,946.0
146670,947,950.0,948.0,947.0,950.0,952.0,946.0,951.0,946.0,947.0
...,...,...,...,...,...,...,...,...,...,...
12224,50,50.0,48.0,49.0,48.0,52.0,52.0,49.0,303.0,304.0
164438,4207,753.0,753.0,52.0,50.0,4207.0,3346.0,3359.0,3346.0,38.0
12221,52,3346.0,784.0,784.0,3346.0,979.0,3324.0,7330.0,3594.0,3329.0


In [6]:
#test_sessions 

In [7]:
%%time 
def prepare_sparse_data(df_csv, dic_pkl):
    df_csv.fillna(0, inplace = True)
    df_csv[sites] = df_csv[sites]
   # df_csv[times] = df_csv[times].astype('datetime64')
    df_csv.fillna(0, inplace = True)
    with open(dic_pkl, "rb") as input_dict:
        site_dict=pickle.load(input_dict)
    name_site = {val:key for (key, val) in site_dict.items()}
    name_site[0] = 'unknown'
    df_csv = df_csv[sites].apply(lambda row: ' '.join([name_site[i] for i in row]), axis=1).tolist()
    
    return df_csv


Wall time: 0 ns


In [8]:
test_sessions  = prepare_sparse_data(test_sessions,  
                      "catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/site_dic.pkl")  

In [9]:
train_sessions = prepare_sparse_data(train_sessions,
                         "catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/site_dic.pkl") 

In [51]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1, 5), max_features=50000)#, tokenizer=(lambda s: s.split()))
#vectorizer = TfidfVectorizer(ngram_range=(1, 5),
#                               max_features=80000) #0.87
#vectorizer = TfidfVectorizer(ngram_range=(1, 3),
#                               max_features=80000, tokenizer=(lambda s: s.split())) 0.48
#vectorizer = TfidfVectorizer(ngram_range=(1, 3),
#                               max_features=50000, tokenizer=(lambda s: s.split())) 0.48
#vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train_sessions)
X_test = vectorizer.transform(test_sessions)
y_train = y.astype('int').values


Wall time: 1min 51s


In [52]:
X_test.shape

(82797, 50000)

In [54]:
%%time
#19-02-2019
def prepare_df_set_features(df_csv, df_times, X_sparse):
        
    times = ['time%s' % i for i in range(1, 11)]
    sites = ['site%s' % i for i in range(1, 11)]   
    print(X_sparse.shape)
    #year
    year = df_times['time1'].apply(lambda x: int(x.strftime("%Y"))).astype('int').values.reshape(-1, 1)
    day = df_times['time1'].apply(lambda x: int(x.strftime("%d"))).astype('int').values.reshape(-1, 1)
    month = df_times['time1'].apply(lambda x: int(x.strftime("%m"))).astype('int').values.reshape(-1, 1)
 #   year1_4 = (month <= 3).astype('int').reshape(-1, 1)
 #   year2_4 = ((month > 3) & (month <= 6)).astype('int').reshape(-1, 1)
 #   year3_4 = ((month > 6) & (month <= 9)).astype('int').reshape(-1, 1)
 #   year4_4 = (month < 9).astype('int').reshape(-1, 1)
#   print("day ", day.shape)
    #start_hour
    start_hour = df_times['time1'].apply(lambda x: int(x.strftime("%H"))).astype('int').values.reshape(-1, 1)
#    print("start_hour  ", start_hour.shape)
   # train_df['start_hour'] = train_df['time1'].apply(lambda ts: ts.hour)
    
#    sess_duration = (df_times.max(axis=1) - df_times.min(axis=1)).astype('timedelta64[s]')\
#            .astype('int').values.reshape(-1, 1)
    #day_of_week
    day_of_week = df_times['time1'].apply(lambda x: int(x.strftime("%u"))).astype('int').values.reshape(-1, 1)
 #  print("day_of_week  ", day_of_week.shape)
    weekday = (day_of_week >= 6).astype('int').reshape(-1, 1)
#   print("weekday  ", weekday.shape)
    workday = (day_of_week < 6).astype('int').reshape(-1, 1)
#   print("workday  ", workday.shape)

  #  day_of_year = df_times['time1'].apply(lambda x: int(x.strftime("%j"))).astype('int').values.reshape(-1, 1)
#   print("day_of_year  ", day_of_year.shape)
        #day of months                               
    day_of_months = df_times['time1'].apply(lambda x: int(x.strftime("%d"))).values.reshape(-1, 1)

    day_of_months_1_2 = (day_of_months <= 15).astype('int').reshape(-1, 1)

    day_of_months_2_2 = (day_of_months <= 16).astype('int').reshape(-1, 1)
    
    
   # df_csv.fillna(0, inplace = True)
     
    hour = df_times['time1'].apply(lambda ts: ts.hour).astype('int').values.reshape(-1, 1)
#    print("hour  ", hour.shape) 
    morning_h = ((hour >= 7) & (hour <= 11)).astype('int').reshape(-1, 1)
#    print("morning_h  ", morning_h.shape) 
    day_h = ((hour >= 12) & (hour <= 18)).astype('int').reshape(-1, 1)
#    print("day_h  ", day_h.shape) 
    evening_h = ((hour >= 19) & (hour <= 23)).astype('int').reshape(-1, 1)
#    print("evening_h  ", evening_h .shape)
    night_h = ((hour >= 0) & (hour <= 6)).astype('int').reshape(-1, 1)
#    print("night_h  ", night_h.shape) 
    
    

    X = np.hstack([ year, month, day,  start_hour,  weekday, day_of_months, workday, hour, morning_h, day_h, evening_h])
  #  X = np.concatenate((year, day, start_hour,  day_of_week, weekday, workday,
   #       day_of_months, day_of_months_1_2, day_of_months_2_2, hour, morning_h, day_h, evening_h, night_h, X_sparse), axis=1)
    #    print("X", X)
    return X

X_fit_test = prepare_df_set_features(test_df, test_times, X_test)
X_fit_test.shape 

(82797, 50000)
Wall time: 13.5 s


(82797, 11)

In [55]:
X_test_all = csr_matrix(hstack([X_test, X_fit_test]))
#X = np.hstack([csr_matrix(X_test), X_fit_test])

In [56]:
%%time
X_fit_train = prepare_df_set_features(train_df, train_times,  X_train)
X_train_all= csr_matrix(hstack([X_train, X_fit_train]))
X_train_all.shape

(253561, 50000)
Wall time: 42.9 s


(253561, 50011)

# linear (logit_grid_searcher )

In [57]:
%%time

#with timer('Cross-validation'):
time_split = TimeSeriesSplit()
logit = LogisticRegression(random_state=7, solver='liblinear',  class_weight= 'balanced')
c_values=np.arange(5,6, 0.1)
#range(1, 10, 10)
#c_values = np.logspace(0, 1, 10)

#'penalty':['l1','l2']
logit_grid_searcher = GridSearchCV(estimator=logit, param_grid={'C': [5.5]},
                                  scoring='roc_auc',  cv=time_split, verbose=1)
logit_grid_searcher.fit(X_train_all, y)
print('CV score', logit_grid_searcher.best_score_)


test_pred = logit_grid_searcher.predict_proba(X_test_all)[:, 1]
pred_df = pd.DataFrame(test_pred, index=np.arange(1, test_pred.shape[0] + 1),
                       columns=['target'])
pred_df.to_csv(f'submission_lg_{1}.csv', index_label='session_id')

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  5.1min finished


CV score 0.9064983236097829
Wall time: 9min 5s


In [58]:
logit_grid_searcher.best_estimator_

LogisticRegression(C=5.5, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=7, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
 X = np.hstack([ year, month, day,  start_hour,  weekday, 
       workday, hour, morning_h, day_h, evening_h])
CV score 0.9108495258250623
Wall time: 3min    
LogisticRegression(C=5.5, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=7, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
X = np.hstack([ year, month, day,  start_hour,  weekday, day_of_months, day_of_months_1_2,
       day_of_months_2_2,  workday, hour, morning_h, day_h, evening_h])

CV score 0.9091891916048932
Wall time: 4min 2s
vectorizer = TfidfVectorizer(ngram_range=(1, 5), max_features=80000, tokenizer=(lambda s: s.split()))

LogisticRegression(C=5.5, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=7, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

# linear (logit)

In [42]:
%%time
#with timer('Cross-validation'):
#time_split = TimeSeriesSplit()
logit = LogisticRegression(random_state=7, 46C=[5.5], solver='liblinear',penalty='l1', class_weight= 'balanced')
#c_values=range(1, 10, 1)
c_values = 3,59
#np.logspace(0, 1, 10)

#'penalty':['l1','l2']
#logit_grid_searcher = GridSearchCV(estimator=logit, param_grid={'C': c_values},
#                                  scoring='roc_auc',  cv=time_split, verbose=1)
logit.fit(X_train_all, y)
print('CV score', logit.score(X_train_all, y))


test_pred = logit.predict_proba(X_test_all)[:, 1]
pred_df = pd.DataFrame(test_pred, index=np.arange(1, test_pred.shape[0] + 1),
                       columns=['target'])
pred_df.to_csv(f'submission_l_{1}.csv', index_label='session_id')

ValueError: Penalty term must be positive; got (C=[5.5])

In [ ]:
#Определение влияния признаков
from sklearn.preprocessing import StandardScaler

clf = LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=7, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)
clf.fit(X_fit_train, y)
#print(np.hstack([np.array(independent_columns_names).reshape(7,1),clf.coef_.reshape(7,1)]))
clf.coef_

In [ ]:
X = np.hstack([ year, day, month, year1_4, year2_4, year3_4, year4_4 , start_hour, sess_duration, 
                   day_of_week, weekday, workday, day_of_year, day_of_months, day_of_months_1_2, 
                   day_of_months_2_2, hour, morning_h, day_h , evening_h, night_h])

clf.coef_:
array([[-1.67226036e-03, -1.54068005e-05, -3.62233605e-06,
        -5.26207369e-07, -1.30127047e-07, -4.77259197e-09,
        -6.57561572e-07, -9.96692728e-06, -2.14024312e-09,
        -2.76419036e-06, -7.84659390e-08, -7.51966891e-07,
        -9.98491908e-05, -1.54068005e-05, -2.62131883e-07,
        -2.80592649e-07, -9.96692728e-06, -4.31354515e-07,
        -3.77801833e-07, -2.12764825e-08,  0.00000000e+00]])

In [ ]:
array([[-4.35647586e-03,  5.15765659e-03, -9.30459002e-03,
        -3.94171496e-01, -4.93501872e-02,  2.19233947e+00,
        -7.60439582e-01,  3.12826781e-01, -3.87077760e-01,
        -2.89894727e-01, -1.89741961e-03,  5.15765659e-03,
        -6.12239996e-01,  1.93821477e+00, -1.32580590e+00]])